# Install & load libraries

To run this jupyter notebook:

* create a virtual environment, for example using [miniconda](https://docs.conda.io/projects/conda/en/latest/user-guide/install/index.html)
* install python 3.8 (so we all have the same version) into your virtual environment (if you have `miniconda` installed, you can do this while setting up the virtual environment: `conda create -n venv_name python=3.8 pip`
* activate virtual environment `conda activate venv_name`
* install necessary packages into the environment (including `jupyter`). Before you install the `spacy` package, check the [documentation](https://spacy.io/usage) and make sure you select the installation method for your operating system. 
* after installing `spacy`, you need to download the transformer pipeline (in English) (this contains the transformer, tagger, parser, ...) - this should be something like `python -m spacy download en_core_web_trf`
* run the notebook: `jupyter notebook`

In [1]:
# !pip install pysrt
# !pip install -U pip setuptools wheel
# !pip install -U spacy
# !python -m spacy download en_core_web_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.4/104.4 kB 1.9 MB/s eta 0:00:000:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 4.2 MB/s eta 0:00:008.8 MB/s eta 0:00:01
  Created wheel for pysrt: filename=pysrt-1.1.2-py3-none-any.whl size=13444 sha256=d08cbdcc2187bebd9c5d3c94053192023ddbef3f5b8e704315877e14849ff194
  Stored in directory: /Users/lenamangold/Library/Caches/pip/wheels/30/a6/ab/4705174e11f44e74d58c14b32edbacbc852644f86658316aef
Successfully built pysrt
  Using cached setuptools-62.6.0-py3-none-any.whl (1.2 MB)
  Attempting uninstall: setuptools
    Found existing installation: setuptools 61.2.0
    Uninstalling setuptools-61.2.0:
      Successfully uninstalled setuptools-61.2.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 9.0 MB/s eta 0:00:00m eta 0:00:010:00:01m
  Attempting uninstall: spacy
    Found existing installation: spacy 3.3.0
    Uninstalling spacy-3.3.0:
      Successfully uninstalled spac

  Using cached torch-1.11.0-cp38-none-macosx_11_0_arm64.whl (43.1 MB)
  Using cached spacy_alignments-0.8.5-cp38-cp38-macosx_11_0_arm64.whl (302 kB)
  Using cached transformers-4.19.4-py3-none-any.whl (4.2 MB)
  Using cached filelock-3.7.1-py3-none-any.whl (10 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.5/101.5 kB 1.5 MB/s eta 0:00:00 MB/s eta 0:00:01
  Using cached PyYAML-6.0-cp38-cp38-macosx_11_0_arm64.whl
  Using cached regex-2022.6.2-cp38-cp38-macosx_11_0_arm64.whl (281 kB)
  Using cached tokenizers-0.12.1.tar.gz (220 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for tokenizers (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [51 lines of output]
      running bdist_wheel
      running build
      running build_py
      creating build
      creating build/lib.macosx-11.1-arm64-cpython-38
      creating bui

In [3]:
import pysrt
import spacy
from tqdm.auto import tqdm
from copy import deepcopy
from tabulate import tabulate

In [4]:
subtitles = pysrt.open("./Copy of 08. Lords of the air.srt")
for subtitle in subtitles[:5]:
    print(subtitle)

1
00:01:19,727 --> 00:01:23,356
White storks. If you wanted to pick one bird

2
00:01:23,567 --> 00:01:26,400
as a representative
of all the birds in the world,

3
00:01:26,447 --> 00:01:29,200
you could do worse
than pick the white stork.

4
00:01:29,327 --> 00:01:32,637
It's a marvellous flyer, an intrepid traveller.

5
00:01:33,167 --> 00:01:38,366
This pair will have come from Africa
to nest in this small town in Bavaria.



Load the transformer model into SpaCy.

In [5]:
# spacy.require_gpu() # Infer through transformer on GPU (far more efficient than on CPU)
nlp = spacy.load("en_core_web_lg") # Load the transformer model into SpaCy

In [6]:
def token_filter(t, pos_filter={}):
    """ 
    Returns a boolean whether a token should be kept or not.
    
    Parameters:
    t (spacy.Doc.Token): A spaCy token from the document being preprocessed.
    
    Returns:
    bool: A boolean to indicate whether the token should be kept or not.
    
    """
    has_digit = lambda s : any(i.isdigit() for i in s)
    return (not t.is_punct 
            and not has_digit(t.text) 
            and len(t.text)>2 
            and not t.is_stop
            and not t.pos_ in pos_filter)

def preprocess(doc, pos_filter={}):
    """
    Splits documents into tokens, filters out unwanted tokens and lemmatizes the
    text.
    
    Parameters:
    doc (spacy.document)
    """
    out = list()
    for subtitle in doc:
        s = []
        for token in subtitle:
            if token_filter(token, pos_filter):
                s.append(token.text.lower())
        out.append(s)
    return out

Tokenise and preprocess. In this example, we only split each line from the subtitles into its individual tokens (see here for details on tokenization) through the `pipe` method. 

The preprocess method removes punctuation, removes words of length 2 or shorter and removes English stopwords (and, of, if, ...).

In [7]:
utterances = [subtitle.text for subtitle in subtitles]

corpus = nlp.pipe(utterances, batch_size=26000)

processed = preprocess(corpus)

processed[:15]

[['white', 'storks', 'wanted', 'pick', 'bird'],
 ['representative', 'birds', 'world'],
 ['worse', 'pick', 'white', 'stork'],
 ['marvellous', 'flyer', 'intrepid', 'traveller'],
 ['pair', 'come', 'africa', 'nest', 'small', 'town', 'bavaria'],
 ['complicated', 'courtship', 'greeting', 'rituals'],
 ['devoted', 'parents'],
 ['stand', 'birds', 'world', 'stork', 'feather'],
 ['seen', 'key', 'crucial', 'bird'],
 ['feather', 'marvellous', 'aerofoil'],
 ['man', 'invent', 'strong', 'weight', 'weight'],
 ['extremely', 'efficient', 'insulator', 'important', 'bird'],
 ['complicated', 'structure'],
 ['feather', 'separate', 'filaments'],
 ['central', 'quill']]

In [8]:
preprocessed_subtitles = deepcopy(subtitles)

for subtitle, proc_subtitle in zip(preprocessed_subtitles, processed):
    subtitle.text = " ".join(proc_subtitle)

In [9]:
for subtitle in preprocessed_subtitles[:5]:
    print(subtitle)

1
00:01:19,727 --> 00:01:23,356
white storks wanted pick bird

2
00:01:23,567 --> 00:01:26,400
representative birds world

3
00:01:26,447 --> 00:01:29,200
worse pick white stork

4
00:01:29,327 --> 00:01:32,637
marvellous flyer intrepid traveller

5
00:01:33,167 --> 00:01:38,366
pair come africa nest small town bavaria



In [10]:
preprocessed_subtitles.save('./preprocessed_sub.txt')